In [14]:
import numpy as np
import torch
import torch.nn.functional as F

In [70]:
def conv3d(input, weight, bias=None, stride=1, padding=0):
    # Получение размеров входных данных
    batch_size, in_channels, in_depth, in_height, in_width = input.shape
    out_channels, _, kernel_depth, kernel_height, kernel_width = weight.shape

    # Вычисление размеров выходных данных после свертки
    out_depth = (in_depth + 2 * padding - kernel_depth) // stride + 1
    out_height = (in_height + 2 * padding - kernel_height) // stride + 1
    out_width = (in_width + 2 * padding - kernel_width) // stride + 1

    # Создание массива для хранения выходных данных
    output = torch.zeros((batch_size, out_channels, out_depth, out_height, out_width))

    # Применение свертки
    for b in range(batch_size):
        for o_c in range(out_channels):
            for d in range(out_depth):
                for h in range(out_height):
                    for w in range(out_width):
                        # Вычисление координат для текущего окна
                        start_d = d * stride
                        end_d = start_d + kernel_depth
                        start_h = h * stride
                        end_h = start_h + kernel_height
                        start_w = w * stride
                        end_w = start_w + kernel_width

                        # Получение окна из входных данных
                        window = input[b, :, start_d:end_d, start_h:end_h, start_w:end_w]

                        # Вычисление свертки в текущем окне
                        if bias is None:
                            output[b, o_c, d, h, w] = torch.sum(window * weight[o_c, :, :, :, :])
                        else:
                            output[b, o_c, d, h, w] = torch.sum(window * weight[o_c, :, :, :, :]) + bias[o_c]

    return output

In [75]:
def test_1():
    # Создание простого объемного изображения (трехмерной матрицы) для свертки
    volume = np.array([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]],
                       [[10, 11, 12],
                        [13, 14, 15],
                        [16, 17, 18]],
                       [[19, 20, 21],
                        [22, 23, 24],
                        [25, 26, 27]]])
    # Простое объемное ядро (фильтр) для свертки
    kernel = np.array([[[1, 0, -1],
                        [1, 0, -1],
                        [1, 0, -1]],
                       [[1, 0, -1],
                        [1, 0, -1],
                        [1, 0, -1]],
                       [[1, 0, -1],
                        [1, 0, -1],
                        [1, 0, -1]]])
    # Преобразование данных в тензоры PyTorch
    volume_tensor = torch.from_numpy(volume).unsqueeze(0).unsqueeze(0).float()  # Добавление размерностей для батча и каналов
    kernel_tensor = torch.from_numpy(kernel).unsqueeze(0).unsqueeze(0).float()  # Добавление размерностей для каналов
    # Ожидаемый результат при использовании torch.nn.functional.conv3d
    expected_result_pytorch = F.conv3d(volume_tensor, kernel_tensor, padding=0)
    # Ожидаемый результат при использовании вашего метода
    expected_result_custom = conv3d(volume_tensor, kernel_tensor)
    # Сравнение результатов
    if torch.allclose(expected_result_pytorch, expected_result_custom):
        print("Результаты совпадают!")
    else:
        print("Результаты не совпадают")

def test_2():
    # Создание объемного изображения (трехмерной матрицы) для свертки
    volume = np.random.rand(2, 3, 4, 4, 4)  # Произвольные данные объема (batch_size, in_channels, depth, height, width)
    
    # Произвольное объемное ядро (фильтр) для свертки
    kernel = np.random.rand(4, 3, 3, 3, 3)  # Произвольные данные ядра (out_channels, in_channels, kernel_depth, kernel_height, kernel_width)
    
    # Преобразование данных в тензоры PyTorch
    volume_tensor = torch.from_numpy(volume).float()
    kernel_tensor = torch.from_numpy(kernel).float()
    
    # Ожидаемый результат при использовании torch.nn.functional.conv3d
    expected_result_pytorch = F.conv3d(volume_tensor, kernel_tensor, padding=0)
    
    # Ожидаемый результат при использовании вашей функции
    expected_result_custom = conv3d(volume_tensor, kernel_tensor)
    
    # Сравнение результатов
    if torch.allclose(expected_result_pytorch, expected_result_custom):
        print("Результаты совпадают!")
    else:
        print("Результаты не совпадают")

def test_3():
    # Создание объемного изображения (трехмерной матрицы) для свертки
    volume = np.random.rand(1, 1, 5, 5, 5)  # Произвольные данные объема (batch_size, in_channels, depth, height, width)
    
    # Произвольное объемное ядро (фильтр) для свертки
    kernel = np.random.rand(1, 1, 3, 3, 3)  # Произвольные данные ядра (out_channels, in_channels, kernel_depth, kernel_height, kernel_width)
    
    # Преобразование данных в тензоры PyTorch
    volume_tensor = torch.from_numpy(volume).float()
    kernel_tensor = torch.from_numpy(kernel).float()
    
    # Ожидаемый результат при использовании torch.nn.functional.conv3d
    expected_result_pytorch = F.conv3d(volume_tensor, kernel_tensor, padding=0)
    
    # Ожидаемый результат при использовании вашей функции
    expected_result_custom = conv3d(volume_tensor, kernel_tensor)
    
    # Сравнение результатов
    if torch.allclose(expected_result_pytorch, expected_result_custom):
        print("Результаты совпадают!")
    else:
        print("Результаты не совпадают")

test_1()
test_2()
test_3()

Результаты совпадают!
Результаты совпадают!
Результаты совпадают!
